In [1]:
import warnings
import astropy.units as u
from zoneinfo import ZoneInfo
from astropy.time import Time
from astroplan import Observer
from datetime import datetime, timezone
from astropy.coordinates import EarthLocation, Angle

# Remove warnings
warnings.filterwarnings('ignore')

# Observation night date
observation_night = '2023-07-10'
local_midnight = True

In [2]:
# T80-South location info
T80S_TZ = ZoneInfo('America/Santiago')
UTC_TZ = timezone.utc
T80S_LAT = '-30d10m04.31s'
T80S_LON = '-70d48m20.48s'
T80S_HEI = 2178
t80s_lat = Angle(T80S_LAT, 'deg')
t80s_lon = Angle(T80S_LON, 'deg')
t80s_hei = T80S_HEI*u.m
t80s_EL = EarthLocation(lat=t80s_lat, lon=t80s_lon, height=t80s_hei)

# Date and time config
dt_obs = datetime.fromisoformat(observation_night +' 23:59:59.000+00:00')
t80s_dt = dt_obs.astimezone(T80S_TZ)
t80s_Time = Time(t80s_dt - t80s_dt.utcoffset(), location=t80s_EL)

# Observer at T80-South location
t80s_obs = Observer(location=t80s_EL, timezone=T80S_TZ)

# Nearest evening and morning twilights timestamp and LST 
twilight_morning = t80s_obs.twilight_morning_astronomical(t80s_Time, which='nearest')
twilight_evening = t80s_obs.twilight_evening_astronomical(t80s_Time, which='nearest')
lst_even = t80s_obs.local_sidereal_time(twilight_evening)
lst_morn = t80s_obs.local_sidereal_time(twilight_morning)

# Night duration (time between twilights)
night_duration = (twilight_morning.datetime - twilight_evening.datetime).total_seconds()/3600

# Moon at midnight
if local_midnight:
    Time_mn = t80s_Time
else:
    Time_mn = t80s_obs.midnight(t80s_Time)
moon_illum = t80s_obs.moon_illumination(Time_mn)
lst_mn = t80s_obs.local_sidereal_time(Time_mn)

In [3]:
print(f'Evening twilight:')
print(f'\tDATETIME (UTC):', twilight_evening.datetime)
print(f'\tLST:', lst_even)
print(f'Morning twilight:')
print(f'\tDATETIME (UTC):', twilight_morning.datetime)
print(f'\tLST:', lst_morn)
if local_midnight:
    print('Using local midnight:')
else:
    print(f'Sun midnight:')
    print(f'\tDATETIME (UTC):', Time_mn.datetime)
    print(f'\tLST:', lst_mn)
print(f'\tMoon illumination: {moon_illum:.4f}')

Evening twilight:
	DATETIME (UTC): 2023-07-10 23:22:57.405336
	LST: 13h54m13.43870452s
Morning twilight:
	DATETIME (UTC): 2023-07-11 10:14:18.775815
	LST: 0h47m21.81025691s
Using local midnight:
	Moon illumination: 0.3834
